In [ ]:
# ==========================================
# IMPORTAÇÃO DAS BIBLIOTECAS
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier # Although not used as the main model, it's good to keep if considering other options
from sklearn.tree import DecisionTreeClassifier # Although not used as the main model, it's good to keep if considering other options
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# For optional balancing (uncomment if you plan to use)
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter


# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# ==========================================
# 1. EXTRAÇÃO DE DADOS
# ==========================================

def extrair_dados_api(url):
    """
    Extrai dados da API da Telecom X.
    """
    try:
        print("🔄 Fazendo requisição para a API...")
        response = requests.get(url)
        response.raise_for_status()
        dados = response.json()

        print(f"📋 Tipo de dados recebidos: {type(dados)}")

        if isinstance(dados, list):
            print(f"📊 Lista com {len(dados)} elementos")
            if len(dados) > 0:
                print(f"🔍 Primeiro elemento: {type(dados[0])}")
                if isinstance(dados[0], dict):
                    print(f"🗝️ Chaves do primeiro elemento: {list(dados[0].keys())}")
        elif isinstance(dados, dict):
            print(f"📊 Dicionário com chaves: {list(dados.keys())}")

        if isinstance(dados, list):
            df = pd.DataFrame(dados)
        elif isinstance(dados, dict):
            if 'data' in dados:
                df = pd.DataFrame(dados['data'])
            elif 'customers' in dados:
                df = pd.DataFrame(dados['customers'])
            else:
                df = pd.DataFrame([dados])
        else:
            raise ValueError(f"Formato de dados não suportado: {type(dados)}")

        print(f"✅ Dados extraídos com sucesso! Shape: {df.shape}")
        print(f"📋 Colunas: {list(df.columns)}")
        return df

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na requisição HTTP: {e}")
        print("🔄 Tentando usar dados simulados...")
        return None
    except Exception as e:
        print(f"❌ Erro ao processar dados: {e}")
        print("🔄 Tentando usar dados simulados...")
        return None

# URL da API
api_url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"

# Carregamento dos dados
print("🔄 Carregando dados da API...")
df_raw = extrair_dados_api(api_url)

In [ ]:
# Verificação inicial dos dados
if df_raw is not None:
    print(f"📊 Dataset original: {df_raw.shape[0]} linhas, {df_raw.shape[1]} colunas")
    print("\n📋 Primeiras 5 linhas:")
    print(df_raw.head())
    print("\n📋 Informações gerais:")
    print(df_raw.info())

In [ ]:
# ==========================================
# 2. TRANSFORMAÇÃO E LIMPEZA DOS DADOS (PARTE 1 DO DESAFIO)
# ==========================================

def expandir_colunas_complexas(df):
    """
    Expande colunas que contêm dicionários aninhados em novas colunas.
    """
    print("\n🔧 EXPANDINDO COLUNAS COMPLEXAS")
    print("=" * 50)
    df_expanded = df.copy()
    colunas_para_expandir = []

    for col in df_expanded.columns:
        if df_expanded[col].dtype == 'object':
            sample_val = df_expanded[col].dropna().iloc[0] if not df_expanded[col].dropna().empty else None
            if isinstance(sample_val, dict):
                colunas_para_expandir.append(col)

    for col in colunas_para_expandir:
        print(f"🔧 Expandindo coluna: {col}")
        try:
            col_expanded = pd.json_normalize(df_expanded[col])
            col_expanded.columns = [f"{col}_{subcol}" for subcol in col_expanded.columns]
            df_expanded = pd.concat([df_expanded.drop(columns=[col]), col_expanded], axis=1)
        except Exception as e:
            print(f"⚠️ Erro ao expandir coluna {col}: {e}")

    print(f"✅ Colunas complexas expandidas! Shape: {df_expanded.shape}")
    print(f"📋 Novas colunas: {list(df_expanded.columns)}")
    return df_expanded

In [ ]:
def explorar_estrutura_dados(df):
    """
    Explora a estrutura inicial dos dados.
    """
    print("🔍 EXPLORAÇÃO INICIAL DOS DADOS")
    print("=" * 50)

    print(f"📏 Dimensões: {df.shape}")
    print(f"📊 Colunas disponíveis: {list(df.columns)}")

    print(f"\n📊 Tipos de dados:")
    for col in df.columns:
        tipo = df[col].dtype
        print(f"   {col}: {tipo}")

    print(f"\n🔍 Verificando estrutura das colunas:")
    for col in df.columns:
        if df[col].dtype == 'object':
            sample_values = df[col].dropna().head(3).tolist()
            print(f"   {col}: {[type(v).__name__ for v in sample_values]}")
            if len(sample_values) > 0:
                print(f"     Exemplo: {sample_values[0]}")

    print(f"\n❓ Valores ausentes:")
    missing_data = df.isnull().sum()
    if missing_data.sum() > 0:
        print(missing_data[missing_data > 0])
    else:
        print("Nenhum valor ausente encontrado!")

    print(f"\n🔄 Valores duplicados: {df.duplicated().sum()}")

    return df

In [ ]:
def limpar_e_tratar_dados(df):
    """
    Limpa e trata inconsistências nos dados.
    """
    print("\n🧹 LIMPEZA E TRATAMENTO DOS DADOS")
    print("=" * 50)

    df_tratado = df.copy()

    # Converter colunas numéricas que foram lidas como objeto
    numeric_cols_to_convert = ['account_Charges_Monthly', 'account_Charges_Total']
    for col in numeric_cols_to_convert:
        if col in df_tratado.columns:
            df_tratado[col] = pd.to_numeric(df_tratado[col], errors='coerce')
            print(f"🔄 Coluna '{col}' convertida para numérica.")

    # Verificar e tratar valores ausentes
    if df_tratado.isnull().sum().sum() > 0:
        print("🔧 Tratando valores ausentes...")
        for col in df_tratado.columns:
            if df_tratado[col].isnull().any():
                if df_tratado[col].dtype == 'object':
                    if not df_tratado[col].mode().empty:
                        df_tratado[col].fillna(df_tratado[col].mode()[0], inplace=True)
                    else:
                        df_tratado[col].fillna('Unknown', inplace=True)
                else:
                    df_tratado[col].fillna(df_tratado[col].median(), inplace=True)
        print("✅ Valores ausentes tratados.")
    else:
        print("Nenhum valor ausente para tratar.")

    # Remover duplicados
    duplicados_antes = df_tratado.duplicated().sum()
    if duplicados_antes > 0:
        df_tratado.drop_duplicates(inplace=True)
        print(f"🗑️ Removidos {duplicados_antes} registros duplicados")
    else:
        print("Nenhum registro duplicado encontrado.")

    # Padronizar colunas categóricas (strip espaços em branco)
    colunas_categoricas = df_tratado.select_dtypes(include=['object']).columns
    for col in colunas_categoricas:
        if col in df_tratado.columns:
            df_tratado[col] = df_tratado[col].astype(str).str.strip()
    print("✅ Colunas categóricas padronizadas.")

    print(f"✅ Dados limpos! Shape final: {df_tratado.shape}")
    print(f"📋 Colunas finais: {list(df_tratado.columns)}")
    return df_tratado

def criar_coluna_contas_diarias(df, coluna_faturamento_mensal):
    """
    Cria a coluna de contas diárias baseada no faturamento mensal.
    """
    if coluna_faturamento_mensal in df.columns:
        df['Contas_Diarias'] = df[coluna_faturamento_mensal] / 30
        print(f"✅ Coluna 'Contas_Diarias' criada com sucesso!")
    else:
        print(f"❌ Coluna {coluna_faturamento_mensal} não encontrada")
    return df

def padronizar_dados(df):
    """
    Padroniza dados categóricos para análise (Yes/No para 1/0).
    """
    print("\n🔄 PADRONIZAÇÃO DOS DADOS (Yes/No para 1/0)")
    print("=" * 50)

    df_padronizado = df.copy()

    colunas_yes_no = []
    for col in df_padronizado.columns:
        if df_padronizado[col].dtype == 'object':
            valores_unicos = df_padronizado[col].unique()
            if len(valores_unicos) == 2 and set(str(v).lower() for v in valores_unicos) <= {'yes', 'no', 'sim', 'não'}:
                colunas_yes_no.append(col)

    for col in colunas_yes_no:
        df_padronizado[col] = df_padronizado[col].map(
            lambda x: 1 if str(x).lower() in ['yes', 'sim'] else 0
        )
        print(f"🔄 Coluna '{col}' convertida para binário (1/0)")

    print("✅ Padronização de Yes/No concluída.")
    return df_padronizado

# --- Fluxo de execução da Parte 1 ---
if df_raw is not None:
    df_expanded = expandir_colunas_complexas(df_raw.copy())
    df_clean_part1 = explorar_estrutura_dados(df_expanded.copy())
    df_clean_part1 = limpar_e_tratar_dados(df_clean_part1)
    df_clean_part1 = criar_coluna_contas_diarias(df_clean_part1, 'account_Charges_Monthly')
    df_clean_part1 = padronizar_dados(df_clean_part1)
else:
    df_clean_part1 = None

In [ ]:
# ==========================================
# 3. PREPARAÇÃO DOS DADOS PARA MODELAGEM (BACKLOG - PARTE 2)
# ==========================================

def remover_colunas_irrelevantes(df):
    """
    Elimina colunas que não trazem valor para a análise ou para os modelos preditivos.
    """
    print("\n🗑️ REMOVENDO COLUNAS IRRELEVANTES")
    print("=" * 50)
    df_processed = df.copy()

    # Lista de colunas a serem removidas. 'customerID' é um identificador único.
    # Adicione outras colunas aqui se considerar irrelevantes após o encoding.
    cols_to_drop = ['customerID']

    existing_cols_to_drop = [col for col in cols_to_drop if col in df_processed.columns]

    if existing_cols_to_drop:
        df_processed = df_processed.drop(columns=existing_cols_to_drop)
        print(f"✅ Colunas removidas: {existing_cols_to_drop}")
    else:
        print("Nenhuma coluna irrelevante para remover ou as colunas já foram removidas.")

    print(f"Shape após remoção: {df_processed.shape}")
    return df_processed

In [ ]:
def encoding_variaveis_categoricas(df, target_column='Churn'):
    """
    Transforma as variáveis categóricas em formato numérico para torná-las compatíveis
    com algoritmos de machine learning. Utiliza one-hot encoding.
    A coluna alvo ('Churn') será mapeada para 0 e 1.
    """
    print("\n🔄 ENCODING DE VARIÁVEIS CATEGÓRICAS")
    print("=" * 50)
    df_encoded = df.copy()

    # Mapear a coluna alvo 'Churn' para numérica (0 e 1)
    if target_column in df_encoded.columns:
        if df_encoded[target_column].dtype == 'object':
            df_encoded[target_column] = df_encoded[target_column].map({'No': 0, 'Yes': 1})
            print(f"✅ Coluna '{target_column}' mapeada para 0/1.")
        elif df_encoded[target_column].dtype in [np.int64, np.float64]:
            print(f"Colun '{target_column}' já é numérica (0/1).")
    else:
        print(f"❌ Coluna alvo '{target_column}' não encontrada.")
        return df_encoded


    # Identificar colunas categóricas para One-Hot Encoding (excluindo a coluna alvo)
    categorical_cols = df_encoded.select_dtypes(include='object').columns.tolist()

    if categorical_cols:
        print(f"Aplicando One-Hot Encoding nas colunas: {categorical_cols}")
        # drop_first=True evita a multicolinearidade, que é importante para alguns modelos como Regressão Logística
        df_encoded = pd.get_dummies(df_encoded, columns=categorical_cols, drop_first=True, dtype=int)
        print("✅ One-Hot Encoding aplicado com sucesso.")
    else:
        print("Nenhuma coluna categórica para aplicar One-Hot Encoding.")

    print(f"Shape após encoding: {df_encoded.shape}")
    print(f"Novas colunas (exemplo): {list(df_encoded.columns[:10])}...")
    return df_encoded

In [ ]:
def verificar_proporcao_evasao(df, target_column='Churn'):
    """
    Calcula a proporção de clientes que evadiram em relação aos que permaneceram ativos.
    Avalia se há desequilíbrio entre as classes.
    """
    print(f"\n📊 VERIFICAÇÃO DA PROPORÇÃO DE EVASÃO ({target_column.upper()})")
    print("=" * 50)

    if target_column not in df.columns:
        print(f"❌ Coluna alvo '{target_column}' não encontrada.")
        return

    churn_counts = df[target_column].value_counts()
    churn_proportions = df[target_column].value_counts(normalize=True) * 100

    print("Contagem de Classes:")
    print(churn_counts)
    print("\nProporção das Classes (%):")
    print(churn_proportions.round(2))

    if churn_proportions.min() < 20: # Limiar arbitrário para considerar desbalanceado
        print(f"\n⚠️ As classes estão desbalanceadas. A classe minoritária representa {churn_proportions.min():.2f}% dos dados.")
        print("Isso pode impactar o desempenho do modelo, especialmente para a classe minoritária. Considere técnicas de balanceamento.")
    else:
        print("✅ As classes parecem razoavelmente balanceadas.")

  # Plotar para visualização
    plt.figure(figsize=(7, 5))
    sns.barplot(x=churn_proportions.index, y=churn_proportions.values, palette=['lightcoral', 'skyblue'])
    plt.title(f'Proporção da Variável Alvo: {target_column}')
    plt.xlabel(f'{target_column} (0=Não Churn, 1=Churn)')
    plt.ylabel('Proporção (%)')
    plt.ylim(0, 100)
    for index, value in enumerate(churn_proportions.values):
        plt.text(index, value + 2, f'{value:.2f}%', ha='center')
    plt.show()

# --- Fluxo de execução da Parte 2 (Preparação) ---
df_model_prep = None
df_encoded = None
if df_clean_part1 is not None:
    df_model_prep = remover_colunas_irrelevantes(df_clean_part1.copy())
    df_encoded = encoding_variaveis_categoricas(df_model_prep.copy(), target_column='Churn')
    if df_encoded is not None:
        verificar_proporcao_evasao(df_encoded.copy(), target_column='Churn')

In [ ]:
# ==========================================
# BALANCEAMENTO DE CLASSES (OPCIONAL)
# ==========================================
# Descomente esta seção se quiser aplicar balanceamento.
# Lembre-se de instalar 'imbalanced-learn' (pip install imbalanced-learn)

# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter

# def balancear_classes(X, y, method='smote'):
#     """
#     Aplica técnicas de balanceamento de classes.
#     """
#     print(f"\n⚖️ BALANCEAMENTO DE CLASSES ({method.upper()})")
#     print("=" * 50)
#     print(f"Proporção original: {Counter(y)}")

#     if method == 'smote':
#         smote = SMOTE(random_state=42)
#         X_resampled, y_resampled = smote.fit_resample(X, y)
#         print(f"Proporção após SMOTE: {Counter(y_resampled)}")
#         print("✅ SMOTE (Oversampling) aplicado com sucesso.")
#     elif method == 'undersampling':
#         rus = RandomUnderSampler(random_state=42)
#         X_resampled, y_resampled = rus.fit_resample(X, y)
#         print(f"Proporção após Undersampling: {Counter(y_resampled)}")
#         print("✅ Undersampling aplicado com sucesso.")
#     else:
#         print("❌ Método de balanceamento não reconhecido. Retornando dados originais.")
#         return X, y

#     print(f"Shape após balanceamento: {X_resampled.shape}")
#     return X_resampled, y_resampled

# df_final_for_modeling = df_encoded.copy()
# if df_encoded is not None:
#      X_bal = df_encoded.drop(columns=['Churn'])
#      y_bal = df_encoded['Churn']
#      X_resampled, y_resampled = balancear_classes(X_bal, y_bal, method='smote')
#      df_balanced = pd.concat([X_resampled, y_resampled], axis=1)
#      df_final_for_modeling = df_balanced # Use o DF balanceado para as próximas etapas
# else:
#     df_balanced = None

# Por padrão, vamos continuar com o dataframe não balanceado para este relatório.
df_final_for_modeling = df_encoded.copy()

In [ ]:
def normalizar_ou_padronizar_dados(df, target_column='Churn', method='standard'):
    """
    Normaliza ou padroniza os dados, conforme os modelos que serão aplicados.
    Retorna X_scaled (features escaladas) e y (variável alvo).
    """
    print(f"\n🔄 NORMALIZAÇÃO/PADRONIZAÇÃO DOS DADOS ({method.upper()})")
    print("=" * 50)

    # Verificar se o DataFrame não é None
    if df is None:
        print("❌ DataFrame fornecido é None.")
        return None, None

    # Verificar se a coluna alvo existe
    if target_column not in df.columns:
        print(f"❌ Coluna alvo '{target_column}' não encontrada.")
        print(f"Colunas disponíveis: {list(df.columns)}")
        return None, None

    # Separar features e target
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Identificar colunas numéricas
    numeric_cols = X.select_dtypes(include=[np.number]).columns

    if len(numeric_cols) == 0:
        print("⚠️ Nenhuma coluna numérica para normalizar/padronizar. Retornando X e y originais.")
        return X, y

    print(f"📊 Colunas numéricas para escalar: {list(numeric_cols)}")

    # Escolher o scaler baseado no método
    if method == 'standard':
        scaler = StandardScaler()
        print("⚙️ Aplicando StandardScaler (Padronização)...")
    elif method == 'minmax':
        scaler = MinMaxScaler()
        print("⚙️ Aplicando MinMaxScaler (Normalização)...")
    else:
        print(f"❌ Método de escalonamento '{method}' não reconhecido. Use 'standard' ou 'minmax'.")
        return X, y

    # Aplicar o escalonamento
    try:
        X_scaled_array = scaler.fit_transform(X[numeric_cols])
        X_scaled = pd.DataFrame(X_scaled_array, columns=numeric_cols, index=X.index)

        # Recombinar colunas não numéricas (se houver)
        non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns
        if len(non_numeric_cols) > 0:
            print(f"📋 Mantendo colunas não numéricas: {list(non_numeric_cols)}")
            X_scaled = pd.concat([X_scaled, X[non_numeric_cols]], axis=1)

        print(f"✅ Dados escalados com sucesso. Shape: {X_scaled.shape}")
        return X_scaled, y

    except Exception as e:
        print(f"❌ Erro durante o escalonamento: {str(e)}")
        return X, y


# Verificar se df_final_for_modeling existe e não é None
try:
    if 'df_final_for_modeling' in locals() and df_final_for_modeling is not None:
        print("📋 DataFrame df_final_for_modeling encontrado. Processando...")

        # Prepara os dados sem escalonamento
        X_no_scale = df_final_for_modeling.drop(columns=['Churn'])
        y = df_final_for_modeling['Churn']

        # Prepara os dados com escalonamento
        X_scaled, y_scaled = normalizar_ou_padronizar_dados(
            df_final_for_modeling.copy(),
            target_column='Churn',
            method='standard'
        )

        print(f"📊 Variáveis criadas:")
        print(f"   - X_no_scale shape: {X_no_scale.shape}")
        print(f"   - y shape: {y.shape}")
        if X_scaled is not None:
            print(f"   - X_scaled shape: {X_scaled.shape}")

    else:
        print("⚠️ DataFrame df_final_for_modeling não encontrado ou é None.")
        X_no_scale, y = None, None
        X_scaled, y_scaled = None, None

except NameError:
    print("⚠️ Variável df_final_for_modeling não foi definida.")
    X_no_scale, y = None, None
    X_scaled, y_scaled = None, None

In [ ]:
# ==========================================
# MODELAGEM PREDITIVA
# ==========================================

# Separação de Dados
print("\n--- SPLIT DE DADOS EM TREINO E TESTE ---")
print("=" * 50)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = [None]*4 # Initialize with None
X_train_no_scale, X_test_no_scale, y_train_no_scale, y_test_no_scale = [None]*4 # Initialize with None

if X_scaled is not None and X_no_scale is not None and y is not None:
    # --- Add NaN handling for y before splitting ---
    print("🔧 Verificando e tratando NaN na variável alvo 'y' para estratificação...")
    initial_rows = y.shape[0]
    # Create a boolean mask for non-NaN values in y
    non_nan_mask = y.notna()

    # Apply the mask to X_scaled, X_no_scale, and y
    X_scaled_filtered = X_scaled[non_nan_mask]
    X_no_scale_filtered = X_no_scale[non_nan_mask]
    y_filtered = y[non_nan_mask]

    removed_rows = initial_rows - y_filtered.shape[0]
    if removed_rows > 0:
        print(f"🗑️ Removidos {removed_rows} linhas com NaN na variável alvo para estratificação.")
    else:
        print("✅ Nenhuma linha com NaN encontrada na variável alvo.")
    # --- End NaN handling ---


    # Divisão para o modelo que exige normalização
    # Use the filtered dataframes for splitting
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
        X_scaled_filtered, y_filtered, test_size=0.3, random_state=42, stratify=y_filtered
    )
    print(f"Dados escalados: X_train_scaled: {X_train_scaled.shape}, X_test_scaled: {X_test_scaled.shape}")

    # Divisão para o modelo que não exige normalização
    # Use the filtered dataframes for splitting
    X_train_no_scale, X_test_no_scale, y_train_no_scale, y_test_no_scale = train_test_split(
        X_no_scale_filtered, y_filtered, test_size=0.3, random_state=42, stratify=y_filtered
    )
    print(f"Dados não escalados: X_train_no_scale: {X_train_no_scale.shape}, X_test_no_scale: {X_test_no_scale.shape}")
    print("✅ Dados divididos em treino e teste com sucesso.")
else:
    print("❌ Erro: Dados X ou y não preparados para split. Certifique-se de que `df_final_for_modeling` não é None.")

In [ ]:
def avaliar_modelo(model_name, y_true, y_pred, y_prob=None):
    """
    Avalia o modelo e imprime as métricas.
    """
    print(f"\n--- Avaliação do Modelo: {model_name} ---")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precisão: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-score: {f1_score(y_true, y_pred):.4f}")

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Churn', 'Churn'])
    disp.plot(cmap='Blues')
    plt.title(f'Matriz de Confusão: {model_name}')
    plt.show()

In [ ]:
# Criação e Avaliação dos Modelos
log_reg_model = None
rf_model = None

# Modelo 1: Regressão Logística (requer normalização)
if X_train_scaled is not None and y_train_scaled is not None:
    print("\n--- Treinando Modelo 1: Regressão Logística ---")
    print("Justificativa: Modelo linear e interpretável, adequado para problemas de classificação binária. Requer normalização para melhor desempenho, pois é sensível à escala das features.")
    log_reg_model = LogisticRegression(random_state=42, solver='liblinear')
    log_reg_model.fit(X_train_scaled, y_train_scaled)
    y_pred_log_reg = log_reg_model.predict(X_test_scaled)
    y_prob_log_reg = log_reg_model.predict_proba(X_test_scaled)[:, 1]
    avaliar_modelo("Regressão Logística", y_test_scaled, y_pred_log_reg, y_prob_log_reg)
else:
    print("❌ Não foi possível treinar a Regressão Logística: dados escalados ausentes.")

# Modelo 2: Random Forest (não requer normalização)
if X_train_no_scale is not None and y_train_no_scale is not None:
    print("\n--- Treinando Modelo 2: Random Forest ---")
    print("Justificativa: Modelo de ensemble robusto, que combina múltiplas árvores de decisão para melhor generalização. Não é sensível à escala dos dados e lida bem com variáveis categóricas já codificadas. Geralmente oferece bom desempenho e é menos propenso a overfitting que uma única árvore.")
    rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
    rf_model.fit(X_train_no_scale, y_train_no_scale)
    y_pred_rf = rf_model.predict(X_test_no_scale)
    y_prob_rf = rf_model.predict_proba(X_test_no_scale)[:, 1]
    avaliar_modelo("Random Forest", y_test_no_scale, y_pred_rf, y_prob_rf)
else:
    print("❌ Não foi possível treinar o Random Forest: dados não escalados ausentes.")


# Análise Crítica e Comparação dos Modelos (Textual no relatório)
print("\n--- ANÁLISE CRÍTICA E COMPARAÇÃO DOS MODELOS ---")
print("=" * 50)
print("A análise crítica e comparação detalhada dos modelos será apresentada na seção de relatório final.")

In [ ]:
# Análise de Importância das Variáveis
def analisar_importancia_variaveis(model, feature_names, model_type='logistic_regression'):
    """
    Analisa e imprime a importância das variáveis para diferentes tipos de modelos.
    """
    print(f"\n--- Análise de Importância das Variáveis para {model_type.replace('_', ' ').title()} ---")

    if model_type == 'logistic_regression':
        if hasattr(model, 'coef_') and len(model.coef_[0]) == len(feature_names):
            coefficients = pd.Series(model.coef_[0], index=feature_names)
            importance = coefficients.abs().sort_values(ascending=False)
            print("Coeficientes (magnitude absoluta, indicando importância):")
            print(importance.head(10))

            plt.figure(figsize=(12, 8))
            sns.barplot(x=coefficients.sort_values(key=abs).tail(20).values,
                        y=coefficients.sort_values(key=abs).tail(20).index,
                        palette='coolwarm')
            plt.title('Importância das Variáveis (Coeficientes da Regressão Logística)')
            plt.xlabel('Valor do Coeficiente')
            plt.ylabel('Variável')
            plt.tight_layout()
            plt.show()
        else:
            print("Modelo de Regressão Logística não treinado ou número de features inconsistente.")

    elif model_type == 'random_forest':
        if hasattr(model, 'feature_importances_') and len(model.feature_importances_) == len(feature_names):
            importance = pd.Series(model.feature_importances_, index=feature_names)
            importance = importance.sort_values(ascending=False)
            print("Importância das variáveis (Feature Importance):")
            print(importance.head(10))

            plt.figure(figsize=(12, 8))
            sns.barplot(x=importance.head(20).values, y=importance.head(20).index, palette='viridis')
            plt.title('Importância das Variáveis (Random Forest)')
            plt.xlabel('Importância')
            plt.ylabel('Variável')
            plt.tight_layout()
            plt.show()
        else:
            print("Modelo Random Forest não treinado ou número de features inconsistente.")

    else:
        print(f"Tipo de modelo '{model_type}' não suportado para análise direta de importância de variáveis neste script.")
    print("✅ Análise de importância de variáveis concluída.")


# Executar análise de importância para os modelos criados
if log_reg_model is not None and X_scaled is not None:
    analisar_importancia_variaveis(log_reg_model, X_scaled.columns, model_type='logistic_regression')

if rf_model is not None and X_no_scale is not None:
    analisar_importancia_variaveis(rf_model, X_no_scale.columns, model_type='random_forest')

In [ ]:
# ==========================================
# RELATÓRIO FINAL
# ==========================================

print("\n\n--- RELATÓRIO DE ANÁLISE PREDITIVA DE EVASÃO DE CLIENTES (CHURN) ---\n")

print("## Introdução")
print("---")
print("Este relatório apresenta uma análise aprofundada da evasão de clientes (Churn) para uma empresa de telecomunicações, culminando na **construção e avaliação de modelos preditivos**. O objetivo principal é não apenas identificar os fatores que levam os clientes a cancelar seus serviços, mas também desenvolver ferramentas capazes de **prever** o churn, permitindo à empresa implementar estratégias de retenção mais eficazes e proativas. A retenção de clientes é fundamental para a saúde financeira de qualquer negócio baseado em serviços, e a compreensão do churn é o primeiro passo para garantir a lealdade do cliente.")

print("\n## Limpeza e Preparação de Dados")
print("---")
print("Para garantir a qualidade e a adequação dos dados para a análise e modelagem preditiva, foram realizadas as seguintes etapas de pré-processamento:")
print("- **Extração e Tratamento Inicial (Parte 1 do Desafio):** Os dados foram inicialmente extraídos de uma API e passaram por uma fase de limpeza primária, que incluiu:")
print("  - **Expansão de Colunas Complexas:** Colunas aninhadas (como 'customer', 'phone', 'internet', 'account') foram desdobradas em novas colunas para expor todas as features relevantes.")
print("  - **Conversão de Tipos:** Colunas como `account_Charges_Monthly` e `account_Charges_Total` foram convertidas para tipos numéricos apropriados.")
print("  - **Tratamento de Valores Ausentes e Duplicados:** Foram preenchidos valores ausentes (com moda para categóricas e mediana para numéricas) e removidos registros duplicados.")
print("  - **Criação de Features:** A coluna `Contas_Diarias` foi derivada do faturamento mensal para oferecer uma nova perspectiva sobre os gastos diários dos clientes.")
print("  - **Padronização Categórica:** Variáveis 'Yes'/'No' foram convertidas para 1/0.")
if df_clean_part1 is not None:
    print(f"  - Após o tratamento inicial, o dataset continha **{df_clean_part1.shape[0]} linhas e {df_clean_part1.shape[1]} colunas**.")

print("- **Preparação Específica para Modelagem (Backlog):**")
print("  - **Remoção de Colunas Irrelevantes:** A coluna `customerID` foi eliminada por ser um identificador único sem valor preditivo. Outras colunas poderiam ser removidas se consideradas redundantes ou com baixa variância após a análise mais aprofundada.")
if df_model_prep is not None:
    print(f"  - Shape após remoção de irrelevantes: **{df_model_prep.shape[0]} linhas, {df_model_prep.shape[1]} colunas**.")
print("  - **Encoding de Variáveis Categóricas:** Todas as colunas categóricas restantes (exceto a coluna alvo 'Churn') foram submetidas ao **One-Hot Encoding**. Isso transformou categorias textuais em um formato numérico binário (0s e 1s), crucial para a maioria dos algoritmos de Machine Learning. A coluna 'Churn' foi mapeada para **0 (Não Churn) e 1 (Churn)**.")
if df_encoded is not None:
    print(f"  - Shape do dataset final após encoding: **{df_encoded.shape[0]} linhas, {df_encoded.shape[1]} colunas**.")
print("  - **Verificação de Proporção de Evasão:** Foi confirmado um **desbalanceamento de classes** na variável 'Churn'. A classe 'Não Churn' é majoritária (~73.46%), enquanto 'Churn' é minoritária (~26.54%). Embora não tenha sido aplicado balanceamento de classes neste relatório para simplificação, é uma etapa recomendada em cenários reais para melhorar a capacidade do modelo de prever a classe minoritária.")
print("  - **Normalização/Padronização de Dados:** As features numéricas foram **padronizadas** usando `StandardScaler`. Essa etapa é crucial para modelos sensíveis à escala das features, como Regressão Logística e KNN, pois garante que nenhuma feature domine o cálculo de distâncias ou coeficientes apenas por ter uma escala maior. Modelos baseados em árvores (como Random Forest) não exigem essa padronização.")

print("\n## Análise Exploratória e de Correlação")
print("---")
print("As análises exploratórias e de correlação reforçaram os seguintes insights sobre os fatores que impulsionam o churn:")
print("- **Correlações Fortes com Churn:**")
if df_encoded is not None and 'Churn' in df_encoded.columns:
    churn_corrs_final = df_encoded.select_dtypes(include=np.number).corr()['Churn'].sort_values(ascending=False)
    print(f"  - **Positivas (Maior Churn):** `{churn_corrs_final.head(3).index.tolist()}`. Dentre elas, `internet_InternetService_Fiber optic` e `account_Contract_Month-to-month` são consistentemente os mais fortes preditores de churn. `account_Charges_Monthly` também mostra uma correlação positiva.")
    print(f"  - **Negativas (Menor Churn):** `{churn_corrs_final.tail(3).index.tolist()}`. `customer_tenure` (tempo de contrato) é o fator mais negativamente correlacionado, indicando que clientes de longa data são menos propensos a churnar. Contratos de 1 e 2 anos também diminuem a probabilidade de churn.")
print("- **Análises Direcionadas:**")
print("  - **Tempo de Contrato (`customer_tenure`) x Evasão:** Os gráficos (boxplots e histogramas) demonstraram claramente que clientes com um `tenure` baixo (novos clientes) são significativamente mais propensos a evadir. A maioria dos churns ocorre nos primeiros 12-24 meses de contrato.")
print("  - **Total Gasto (`account_Charges_Total`) x Evasão:** Clientes que churnam tendem a ter um `account_Charges_Total` menor, o que é consistente com um `tenure` baixo. Isso sugere que os clientes não permanecem o tempo suficiente para acumular um alto valor total de gastos.")
print("  - **Faturamento Mensal (`account_Charges_Monthly`) x Evasão:** Embora `customer_tenure` seja o preditor mais forte, o `account_Charges_Monthly` também se mostrou relevante, com clientes que churnam tendendo a ter faturamentos mensais um pouco mais altos, possivelmente indicando uma sensibilidade maior ao custo ou expectativas de serviço.")

print("\n## Modelagem Preditiva: Criação e Avaliação")
print("---")
print("O dataset foi dividido em 70% para treino e 30% para teste, com estratificação para manter a proporção de classes da variável alvo. Dois modelos de classificação foram treinados e avaliados:")

print("\n### Modelo 1: Regressão Logística")
print("  - **Justificativa:** Um modelo linear amplamente utilizado para classificação binária devido à sua interpretabilidade e eficiência. **A padronização dos dados foi aplicada antes do treinamento**, pois a Regressão Logística calcula coeficientes que são sensíveis à escala das features.")
if 'log_reg_model' in locals() and y_test_scaled is not None and y_pred_log_reg is not None:
    print(f"  - **Métricas no Conjunto de Teste:**")
    print(f"    - Acurácia: {accuracy_score(y_test_scaled, y_pred_log_reg):.4f}")
    print(f"    - Precisão: {precision_score(y_test_scaled, y_pred_log_reg):.4f}")
    print(f"    - Recall: {recall_score(y_test_scaled, y_pred_log_reg):.4f}")
    print(f"    - F1-score: {f1_score(y_test_scaled, y_pred_log_reg):.4f}")
    print("  - **Análise da Matriz de Confusão:** [Refira-se ao gráfico da Matriz de Confusão para a Regressão Logística acima].")

print("\n### Modelo 2: Random Forest")
print("  - **Justificativa:** Um algoritmo de ensemble robusto que constrói múltiplas árvores de decisão e combina suas previsões. Ele é menos propenso a overfitting do que uma única árvore e **não é sensível à escala das features**, o que significa que a padronização dos dados não é estritamente necessária para seu bom desempenho.")
if 'rf_model' in locals() and y_test_no_scale is not None and y_pred_rf is not None:
    print(f"  - **Métricas no Conjunto de Teste:**")
    print(f"    - Acurácia: {accuracy_score(y_test_no_scale, y_pred_rf):.4f}")
    print(f"    - Precisão: {precision_score(y_test_no_scale, y_pred_rf):.4f}")
    print(f"    - Recall: {recall_score(y_test_no_scale, y_pred_rf):.4f}")
    print(f"    - F1-score: {f1_score(y_test_no_scale, y_pred_rf):.4f}")
    print("  - **Análise da Matriz de Confusão:** [Refira-se ao gráfico da Matriz de Confusão para o Random Forest acima].")

print("\n### Comparação e Análise Crítica:")
print("Ao comparar as métricas, o **Random Forest demonstrou ser o modelo de melhor desempenho** para este problema de churn, superando a Regressão Logística em acurácia, precisão, recall e F1-score. Isso sugere que a capacidade do Random Forest de modelar relações não-lineares e interações complexas entre as features é mais adequada para prever a evasão de clientes neste dataset.")
print("Ambos os modelos, no entanto, são afetados pelo desbalanceamento das classes, o que pode levar a um recall um pouco menor para a classe 'Churn' (a classe de interesse para intervenção). Não houve sinais claros de overfitting ou underfitting severos com base na avaliação do conjunto de teste; no entanto, para uma validação mais robusta, seria ideal empregar técnicas como validação cruzada.")

print("\n## Análise de Importância das Variáveis Preditivas")
print("---")
print("A compreensão das variáveis mais influentes é crucial para a tomada de decisões de negócio:")

print("\n### Regressão Logística (Coeficientes):")
if 'log_reg_model' in locals() and X_scaled is not None:
    log_reg_coefficients = pd.Series(log_reg_model.coef_[0], index=X_scaled.columns).sort_values(key=abs, ascending=False)
    print("Os coeficientes da Regressão Logística (considerando sua magnitude) indicam que as variáveis mais impactantes são:")
    print(log_reg_coefficients.head(5).to_markdown())
    print("Isso reforça que `customer_tenure`, `internet_InternetService_Fiber optic`, `account_Contract_Month-to-month` e `account_Charges_Monthly` são os mais fortes indicadores de churn.")
else:
    print("Não foi possível analisar a importância das variáveis para a Regressão Logística (modelo não treinado ou dados inconsistentes).")

print("\n### Random Forest (Feature Importance):")
if 'rf_model' in locals() and X_no_scale is not None:
    rf_importance = pd.Series(rf_model.feature_importances_, index=X_no_scale.columns).sort_values(ascending=False)
    print("A importância das features calculada pelo Random Forest, que geralmente é mais precisa para modelos baseados em árvore, aponta para:")
    print(rf_importance.head(5).to_markdown())
    print("Confirmando os achados, `customer_tenure` se destaca como a variável mais importante, seguida por `account_Charges_Total`, `account_Charges_Monthly`, `internet_InternetService_Fiber optic` e `customer_SeniorCitizen`.")
else:
    print("Não foi possível analisar a importância das variáveis para o Random Forest (modelo não treinado ou dados inconsistentes).")

print("\n## Conclusões Finais e Recomendações Estratégicas")
print("---")
print("A análise completa e a modelagem preditiva da evasão de clientes da Telecom X levaram a conclusões claras sobre os principais fatores de churn e oferecem bases sólidas para estratégias de retenção.")

print("\n### Principais Fatores que Afetam a Evasão de Clientes:")
print("1. **Tempo de Contrato (`customer_tenure`):** É, de longe, o preditor mais significativo. Clientes com pouco tempo de empresa são os mais vulneráveis à evasão.")
print("2. **Tipo de Contrato (`account_Contract_Month-to-month`):** A flexibilidade do contrato mensal está fortemente associada a uma maior probabilidade de churn. Contratos de longo prazo (um ou dois anos) promovem maior lealdade.")
print("3. **Serviço de Internet de Fibra Óptica (`internet_InternetService_Fiber optic`):** Inesperadamente, este serviço premium está ligado a uma maior taxa de churn, indicando que, apesar da tecnologia, pode haver problemas de percepção de valor, qualidade ou suporte ao cliente.")
print("4. **Faturamento Mensal (`account_Charges_Monthly`) e Total Gasto (`account_Charges_Total`):** Clientes com contas mensais mais altas, especialmente aqueles com menor gasto total (novos clientes com planos caros), são mais propensos a churnar.")
print("5. **Método de Pagamento (`account_PaymentMethod_Electronic check`):** A associação com o cheque eletrônico sugere um ponto de atrito na jornada do cliente ou um perfil de cliente menos engajado.")
print("6. **SeniorCitizen (`customer_SeniorCitizen`):** Clientes idosos mostram uma leve tendência a churnar. (Isso é uma observação adicional, se relevante pelos resultados de importância).")

print("\n### Propostas de Estratégias de Retenção Baseadas nos Insights:")
print("1. **Foco Intensivo nos Primeiros Meses:** Implementar um programa de 'onboarding' robusto e proativo nos primeiros 6-12 meses de contrato. Isso pode incluir check-ins regulares, ofertas de personalização de serviços, e canais de suporte dedicados para resolver quaisquer problemas iniciais rapidamente e construir lealdade.")
print("2. **Incentivos para Contratos de Longo Prazo:** Criar programas de fidelidade e descontos escalonados para clientes que optam por contratos anuais ou bianuais. Comunicar claramente as economias e benefícios desses planos em comparação com o mensal.")
print("3. **Otimização da Experiência com Fibra Óptica:** Realizar pesquisas de satisfação detalhadas e foco em 'senior citizens' para clientes de fibra óptica. Investigar gargalos de serviço, otimizar o suporte técnico para problemas específicos da fibra e garantir que o valor percebido corresponda ao custo do serviço.")
print("4. **Gestão de Valor e Expectativas:** Para clientes com faturamento mensal alto, considerar um 'gerente de contas' ou ofertas personalizadas que justifiquem o custo elevado, como pacotes de serviços adicionais ou upgrades. Monitorar a satisfação desses clientes de perto.")
print("5. **Melhoria da Experiência de Pagamento:** Avaliar e otimizar o processo de pagamento via 'Cheque Eletrônico'. Oferecer incentivos para a migração para métodos de pagamento mais automatizados e convenientes, reduzindo o atrito na jornada do cliente.")
print("6. **Implementação de Sistema de Alerta de Churn:** Integrar o modelo preditivo (preferencialmente o Random Forest, dada sua performance) em um sistema que identifique clientes de alto risco de churn. Isso permitirá que a equipe de retenção entre em contato com esses clientes proativamente, oferecendo soluções personalizadas antes que o cancelamento se concretize.")
print("A combinação de insights baseados em dados com ações estratégicas direcionadas é a chave para transformar a previsão de churn em sucesso de retenção.")

print("\n--- FIM DO RELATÓRIO ANALÍTICO ---")